In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import numpy as np
import matplotlib.pyplot as plt
from pympc.models.boxatlas_parametric import BoxAtlas, MovingLimb, FixedLimb
from pympc.dynamical_systems import dare, moas_closed_loop_from_orthogonal_domains
from pympc.control import MPCHybridController
from pympc.geometry.polytope import Polytope
import pympc.plot as mpc_plt
import director.viewerclient as vc
from director.thirdparty import transformations

Numeric parameters

In [3]:
parameters = {
    'mass': 1.,
    'stiffness': 100.,
    'damping': 1000.,
    'friction_coefficient': .5,
    'gravity': 10.,
    'sampling_time': .1
    }

Nominal state and input

In [4]:
weight = parameters['mass'] * parameters['gravity']
penetration = weight / parameters['stiffness']
nominal_limb_positions = {
    'lf': np.array([[.2], [-.5 - penetration / 2.]]),
    'rf': np.array([[-.2], [-.5 - penetration / 2.]]),
    'lh': np.array([[.3], [.0]]),
    'rh': np.array([[-.3], [.0]])
    }
nominal_limb_forces = {
    'lf': np.array([[weight / 2.], [0.]]),
    'rf': np.array([[weight / 2.], [0.]]),
    'lh': np.zeros((2,1)),
    'rh': np.zeros((2,1))
    }

State limits

In [5]:
kinematic_limits = {
        'b': {'min': np.array([[-.2],[-.1]]), 'max': np.array([[.2],[.1]])},
        'lf': {'min': np.array([[.0],[-.7]]), 'max': np.array([[.4],[-.3]])}, # wrt body
        'rf': {'min': np.array([[-.4],[-.7]]), 'max': np.array([[0.],[-.3]])}, # wrt body
        'lh': {'min': np.array([[.2],[-.2]]), 'max': np.array([[.5],[.2]])}, # wrt body
        'rh': {'min': np.array([[-.5],[-.2]]), 'max': np.array([[-.2],[.2]])}, # wrt body
        }
velocity_limits = {
        'b': {'min': -np.ones((2,1)), 'max': np.ones((2,1))},
        'lf': {'min': -np.ones((2,1)), 'max': np.ones((2,1))},
        'rf': {'min': -np.ones((2,1)), 'max': np.ones((2,1))},
        'lh': {'min': -np.ones((2,1)), 'max': np.ones((2,1))},
        'rh': {'min': -np.ones((2,1)), 'max': np.ones((2,1))},
        }

Force limits

In [6]:
f_min = np.array([[0.], [-parameters['friction_coefficient'] * weight]])
f_max = np.array([[weight], [parameters['friction_coefficient'] * weight]])
force_limits = {
        'lf': {'min': f_min*2., 'max': f_max*2.},
        'rf': {'min': f_min*2., 'max': f_max*2.},
        'lh': {'min': f_min, 'max': f_max},
        'rh': {'min': f_min, 'max': f_max},
        }

Robot topology

In [7]:
lh = [
    MovingLimb(np.array([[1., 0.],[0., -1.]]), np.ones((2,1))/2., None),
    MovingLimb(np.array([[-1., 0.],[1., 0.],[0., -1.]]), np.array([[-.5],[.5+penetration],[.5]]), 0)
    ]
rh = [
    MovingLimb(np.array([[0., -1.],[-3., -1.],[-3., 1.]]), np.array([[.5],[1.5],[1.5]]), None),
    MovingLimb(np.array([[0., -1.],[0., 1.],[3., 1.],[-3., -1.]]), np.array([[.5],[0.],[-1.5],[1.5+penetration*np.sqrt(10)]]), 2),
    MovingLimb(np.array([[0., -1.],[3., -1.],[-3., 1.]]), np.array([[0.],[-1.5],[1.5+penetration*np.sqrt(10)]]), 1),
]


lf = FixedLimb(np.array([[.2],[-.5]]), np.array([[0.],[1.]]))
rf = FixedLimb(np.array([[-.2],[-.5]]), np.array([[0.],[1.]]))
topology = { 'moving': {'lh': lh, 'rh': rh}, 'fixed': {'lf': lf, 'rf': rf} }

Construct box atlas

In [8]:
box_atlas = BoxAtlas(
    topology,
    parameters,
    nominal_limb_positions,
    nominal_limb_forces,
    kinematic_limits,
    velocity_limits,
    force_limits
)

In [9]:
print box_atlas.contact_modes
box_atlas.print_state()
box_atlas.print_input()

[{'rh': 0, 'lh': 0}, {'rh': 0, 'lh': 1}, {'rh': 1, 'lh': 0}, {'rh': 1, 'lh': 1}, {'rh': 2, 'lh': 0}, {'rh': 2, 'lh': 1}]
['q_rh_x', 'q_rh_y', 'q_lh_x', 'q_lh_y', 'q_b_x', 'q_b_y', 'v_b_x', 'v_b_x']
['v_rh_x', 'v_rh_y', 'v_lh_x', 'v_lh_y', 'f_lf_n', 'f_lf_t', 'f_rf_n', 'f_rf_t']


Hybrid controller

In [10]:
N = 10
Q = box_atlas.penalize_relative_positions(np.eye(box_atlas.n_x))
R = np.eye(box_atlas.n_u)
objective_norm = 'two'
terminal_mode = 0
P, K = dare(
    box_atlas.affine_systems[terminal_mode].A,
    box_atlas.affine_systems[terminal_mode].B,
    Q,
    R)
X_N = moas_closed_loop_from_orthogonal_domains(
    box_atlas.affine_systems[terminal_mode].A,
    box_atlas.affine_systems[terminal_mode].B,
    K,
    box_atlas.state_domains[terminal_mode],
    box_atlas.input_domains[terminal_mode])
#X_N = Polytope.from_bounds(-np.ones((8,1))/1000., np.ones((8,1))/1000.)
#X_N.assemble()

Computation of MOAS started... MOAS found.
MOAS facets are 228, removing redundant ones... redundant factes removed, MOAS facets are 103.


Hybrid controller

In [11]:
controller = MPCHybridController(box_atlas.pwa_system, N, objective_norm, Q, R, P, X_N)

Closed-loop system simulation

In [ ]:
N_sim = 20
x_0 = np.array([
    [.0],
    [.0],
    [.0],
    [.0],
    [.0],
    [.0],
    [-.9], # v_b_x
    [.0] # v_b_y
])
u = []
x = [x_0]
u_ws = None
x_ws = None
ss_ws = None
for k in range(N_sim):
    u_k, x_k, ss_k = controller.feedforward(x[k], u_ws, x_ws, ss_ws)[0:3]
    print('Time step ' + str(k) + ': ' + str(ss_k) + '.\r'),
    x_next = box_atlas.pwa_system.simulate(x[k], [u_k[0]])[0][1]
    u.append(u_k[0])
    x.append(x_next)
    u_ws = u_k[1:] + [K.dot(x_k[-1])]
    x_ws = x_k[1:] + [box_atlas.pwa_system.simulate(x_k[-1], [u_ws[-1]])[0][1]]
    ss_ws = ss_k[1:] + (terminal_mode,)
mpc_plt.input_sequence(u, parameters['sampling_time'])
plt.show()
mpc_plt.state_trajectory(x, parameters['sampling_time'])
plt.show()

# Visualizer

Visualizer object

In [12]:
vis = vc.Visualizer()['box_altas']
vertical_translation = - (nominal_limb_positions['lf'][1,0] + nominal_limb_positions['rf'][1,0]) / 2.
vis.settransform(vc.transformations.translation_matrix([0.,0.,vertical_translation]))

Numeric parameters

In [13]:
visualizer_parameters = {
    'wall_x': 1.,
    'wall_y': .02,
    'body_xyz': .2,
    'body_color': np.array([0.,0.,1.]),
    'limb_color': np.array([1.,0.,0.]),
    'wall_color': np.array([0.,1.,0.])
}

Body

In [14]:
vis['bq'].setgeometry(
    vc.GeometryData(
        vc.Box(
            lengths = [visualizer_parameters['body_xyz']]*3),
            color = np.hstack((visualizer_parameters['body_color'], 1.))
            )
    )

Limbs

In [15]:
for limb in ['lf', 'rf', 'lh', 'rh']:
    vis[limb].setgeometry(
        vc.GeometryData(
            vc.Sphere(radius = .05),
            color = np.hstack((visualizer_parameters['limb_color'], 1.))
            )
        )

Environment

In [16]:
vis = box_atlas.visualize_environment(vis)

Animation

In [ ]:
box_atlas.visualize(vis, x[0])
time.sleep(10 * parameters['sampling_time'])
for k in range(N_sim):
    box_atlas.visualize(vis, x[k])
    time.sleep(parameters['sampling_time'])

# Library

In [17]:
from pympc.control import FeasibleSetLibrary

In [18]:
# sample filter
state_domains = box_atlas._domains()[0]

In [19]:
# initialization of the library
library = FeasibleSetLibrary(controller)

In [ ]:
# coverage
sample_step = 50
n_steps = 20
for i in range(n_steps):
    library.sample_policy(sample_step, state_domains)
    print(str((i+1)*sample_step) + ' samples added to the library.')
    np.save('library_box_atlas_' + str((i+1)*sample_step), library)

Sample 0: 
solving MIQP...  solution found in 27.9818198681 s.
new switching sequence (0, 2, 0, 0, 0, 0, 0, 0, 0, 0).
condensing QP...  QP condensed in 0.00459480285645 s.
constructing inner simplex...  inner simplex constructed in 5.99279594421 s.
including sample in inner approximation...  sample included in 1.52813315392 s.
Sample 1: 
solving MIQP...  solution found in 0.475937128067 s.
unfeasible.
Sample 2: 
solving MIQP...  solution found in 6.69820189476 s.
new switching sequence (0, 0, 4, 4, 4, 0, 0, 0, 0, 0).
condensing QP...  QP condensed in 0.00472593307495 s.
constructing inner simplex...  inner simplex constructed in 6.66452503204 s.
including sample in inner approximation...  sample included in 5.6945939064 s.
Sample 3: 
solving MIQP...  solution found in 12.4916651249 s.
new switching sequence (0, 1, 0, 0, 0, 0, 0, 0, 0, 0).
condensing QP...  QP condensed in 0.00466704368591 s.
constructing inner simplex...  inner simplex constructed in 5.6619591713 s.
including sample in

In [ ]:
# shifted switching sequences
#library.add_shifted_switching_sequences(terminal_mode)

In [ ]:
# load
#library = np.load('library_cart_pole_100k.npy').item()

# Tuning

In [ ]:
X = box_atlas._state_constraints()
X.assemble()
library = FeasibleSetLibrary(controller)
for i in range(100):
    x = library.random_sample(state_domains)
    box_atlas.visualize(vis, x)
    time.sleep(.3)